In [2]:
from mp_api.client import MPRester
from pymatgen.core import Structure, Lattice, Element, Composition
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.analysis.structure_matcher import StructureMatcher as SM
from pymatgen.analysis.structure_matcher import FrameworkComparator

import os
import re
import sys
import json
import shutil
import numpy as np
from time import time
from glob import glob
from pprint import pprint
from copy import deepcopy
from monty.serialization import dumpfn,loadfn

In [5]:
mpr = MPRester(api_key="dPcAQJZ6y1NZidGuvTerIPPFXHtsOb3E")

In [3]:
def read_json(fjson):
    
    with open(fjson) as f:
        return json.load(f)


def write_json(d, fjson):
    
    with open(fjson, 'w') as f:
        json.dump(d, f)
    return


def get_inputs():

    for mpid in mpid_directory:

        struc = Structure.from_file(os.path.join(dope, 'POSCAR'))
        relax_set = MPRelaxSet(structure=struc)
        relax_set.write_input(output_dir=dope, potcar_spec=False)
        job_path = os.path.join(dope, 'job.sh')
        shutil.copyfile(sample_job, job_path)

    return

In [8]:
AnionLst=['P','S','Si','B','O'];
NotInclude=['H','Li','Be','C','N','F','Cl','Br','I','At','Ts','K','Rb','Cs','Fr','Ra',\
            'Se','Te','Po','Lv','As','Bi','Mc','Pb','Fl','Nh','Tl',\
            'Tc','Ru','Rh','Pd','Ag','Cd','Re','Os','Pt','Au','Hg',\
            'Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn',\
            'Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu',\
            'Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr'];
NotInclude = list(set(NotInclude) - set(['Ag', 'Au', 'Hg', 'Os', 'Pd', 'Pt', 'Re', 'Rh', 'Ru', 'Tc']))
TmInclude = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu',
             'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag',
             'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg']

In [10]:
#TODO: Need to check ICSD tag (or experimentally observed) from materials project data.

EntryDict={};
for NEle in range(4,8):
    with MPRester('dPcAQJZ6y1NZidGuvTerIPPFXHtsOb3E') as mpr: 
        EntryDict[NEle]=mpr.get_entries(chemsys_formula_mpids="Li-O-*",
                                        additional_criteria={"num_elements":NEle},
                                        inc_structure=None);

Retrieving ThermoDoc documents:   0%|          | 0/23407 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/5705 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/35 [00:00<?, ?it/s]

In [43]:
# write_json(EntryDict, '/Users/yun/Berkeley/Codes/berkeley-ai-hack/data/raw_data.json')

In [58]:
###1-2. Exclude compounds that contain elements in NotInclude
NTotal=0;
StrLst=[]
for NEle, Entries in EntryDict.items():
    print('{} Entries found for NEle={}'.format(len(Entries), NEle));
    ValidMaterNum=0
    for EntryInd, Entry in enumerate(Entries):
        Comp=Entry.composition
        EleLst=list(Comp.get_el_amt_dict().keys())
        IsValid=True
        for Ele in EleLst:
            if Ele in TmInclude:
                IsValid=False
                break
        if not IsValid: 
            continue
        new_dict = {}
        new_dict['structure'] = Entry.structure.as_dict()
        new_dict['mpid'] = Entry.entry_id
        StrLst.append(new_dict); # Structure method under computedstructureentry
        ValidMaterNum+=1
    print('{} materials found for NEle={}'.format(ValidMaterNum,NEle))
    NTotal+=ValidMaterNum
print('{} Li Oxide Compounds found'.format(NTotal))

23408 Entries found for NEle=4
2330 materials found for NEle=4
5705 Entries found for NEle=5
665 materials found for NEle=5
1000 Entries found for NEle=6
94 materials found for NEle=6
35 Entries found for NEle=7
10 materials found for NEle=7
3099 Li Oxide Compounds found


In [66]:
write_json(StrLst, '/Users/yun/Berkeley/Codes/berkeley-ai-hack/data/raw_data_Li-O.json')

In [9]:
StrLst = read_json('../data/raw_data_Li-O.json')

In [ ]:
main_dir = '../data'
save_dir = os.path.join(main_dir, 'li_electrolyte_w_description')

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for d in StrLst:

    mpid = '-'.join(d['mpid'].split('-')[:-1])
    dir_name = mpid
    spec_dir = os.path.join(save_dir, dir_name)

    struc = Structure.from_dict(d['structure'])
    relax_set = MPRelaxSet(structure=struc)
    relax_set.write_input(output_dir=spec_dir, potcar_spec=True)
    
    try:
        robocrys = mpr.materials.robocrys.get_data_by_id('-'.join(d['mpid'].split('-')[:-1]))
        description_file = os.path.join(spec_dir, 'robocrys.txt')
        with open(description_file, 'w') as file:
            file.write(robocrys.description)
    except TypeError:
        print(mpid)

In [47]:
inone = {}

target_dict = '../data/li_electrolyte_w_description'

In [53]:
all_mpid_list = glob(target_dict + '/*/')

for mpid_dir in all_mpid_list:
    
    mpid = mpid_dir.split('/')[-2]
    incar = os.path.join(mpid_dir, 'INCAR')
    kpoints = os.path.join(mpid_dir, 'KPOINTS')
    poscar = os.path.join(mpid_dir, 'POSCAR')
    description = os.path.join(mpid_dir, 'robocrys.txt')
    
    with open(incar, 'r') as file:
        incar_content = file.read()
    with open(kpoints, 'r') as file:
        kpoints_content = file.read()
    with open(poscar, 'r') as file:
        poscar_content = file.read()
    with open(description, 'r') as file:
        description_content = file.read()
        
    inone[mpid] = {}
    inone[mpid]['incar'] = incar_content
    inone[mpid]['kpoints'] = kpoints_content
    inone[mpid]['poscar'] = poscar_content
    inone[mpid]['robocrys'] = description_content
    inone[mpid]['structure'] = Structure.from_file(poscar).as_dict()

In [56]:
write_json(inone, '../data/finetune_data.json')

In [49]:
len(inone.keys())

1

In [29]:
all_mpid_list[0].split('/')[-2]

'mp-644256'

In [33]:
mpid_dir

'../data/li_electrolyte_w_description/mp-644256/'

In [46]:
inone.keys()

dict_keys(['mp-644256', 'incar', 'kpoints', 'poscar', 'robocrys', 'structure'])